# Как научить компьютер читать? 

В этой тетрадке мы обучим свой собственный word2vec. Делать мы это будем на каком-нибудь не очень большом тексте, который вам предстоит выбрать самому. На выбор есть [несколько сказок](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/vector-school) и других [литературных штук](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/books_before) из школьной программы. 

In [1]:
!pip install pymorphy2

^C


In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Ссылка на выбранное вами произведение
# Я взял преступление и наказание (ненвижу Достоевского)
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'


Спарсим текст из файлика.

In [6]:
import requests

resp = requests.get(url)
text = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text[-500:])

смотреть на эти семь лет, как на семь
дней. Он даже и не знал того, что новая жизнь не даром же ему достается, что ее надо еще дорого купить, заплатить за нее великим, будущим подвигом...
Но тут уж начинается новая история, история постепенного обновления человека, история постепенного перерождения его, постепенного перехода из одного мира в другой, знакомства с новою, доселе совершенно неведомою действительностью. Это могло бы составить тему нового рассказа, - но теперешний рассказ наш окончен.


## 1. Предобработка

Теперь нам надо его немного предобработать.  Пусть все слова пишутся с маленькой буквы. 

In [5]:
text = text.lower()

Разобьём весь текст на предложения. 

In [1]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)
len(sents)

NameError: name 'text' is not defined

In [7]:
sents[220]

'действительно, на его платье и даже в волосах кое-где виднелись прилипшие былинки сена.'

Разобьём каждое предложение на отдельные слова.

In [8]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
tokenizer.tokenize(sents[220])

['действительно',
 'на',
 'его',
 'платье',
 'и',
 'даже',
 'в',
 'волосах',
 'кое',
 'где',
 'виднелись',
 'прилипшие',
 'былинки',
 'сена']

In [9]:
sents_tokenize = [tokenizer.tokenize(sent) for sent in sents]
sents_tokenize[4:6]

[['и',
  'каждый',
  'раз',
  'молодой',
  'человек',
  'проходя',
  'мимо',
  'чувствовал',
  'какое',
  'то',
  'болезненное',
  'и',
  'трусливое',
  'ощущение',
  'которого',
  'стыдился',
  'и',
  'от',
  'которого',
  'морщился'],
 ['он',
  'был',
  'должен',
  'кругом',
  'хозяйке',
  'и',
  'боялся',
  'с',
  'нею',
  'встретиться']]

In [10]:
# Flatten без numpy :) 
words = [item for sent in sents_tokenize for item in sent]

In [11]:
len(words) # всего слов

173403

In [12]:
len(set(words)) # уникальных слов

24925

Можно выбросить все стоп-слова. 

In [13]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian') 
stopwords_ru[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [14]:
sents_tokenize = [[word for word in text_cur if word not in stopwords_ru] for text_cur in sents_tokenize]

Слов в корпусе не очень много. Давайте лемматизируем их.  В этом нам поможет библиотека **pymorphy2.**

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). [Документация по pymorphy2.](https://pymorphy2.readthedocs.io/en/latest/)

In [15]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

text = "Филипп пошёл в Авеньон и пленил пап!"
tokens = tokenizer.tokenize(text)

" ".join(morph.normal_forms(token)[0] for token in tokens)

'филипп пойти в авеньон и пленить папа'

In [16]:
p = morph.parse('стали')
p

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.975342, methods_stack=((DictionaryAnalyzer(), 'стали', 945, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.010958, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.005479, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 9),))]

Обработаем все слова из датасета. 

In [17]:
sents_tokenize = [[morph.normal_forms(word)[0] for word in text_cur] for text_cur in sents_tokenize]

In [18]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [19]:
len(words) # всего слов

93069

In [20]:
len(set(words)) # уникальных слов

11084

Хватит обработок! Мы тут не анализом текстов занимаемся, а нейросетками. Если хочешь больше предобработки, [читай мой мануал](https://nbviewer.jupyter.org/github/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb) об этом.  Давайте построим словарик с частотностями и перейдём к моделированию. 

In [21]:
from collections import Counter

word_dict = Counter(words)
word_dict.most_common()[:20]

[('это', 1449),
 ('всё', 963),
 ('знать', 630),
 ('раскольник', 567),
 ('свой', 549),
 ('один', 548),
 ('сказать', 544),
 ('говорить', 536),
 ('человек', 501),
 ('весь', 442),
 ('стать', 441),
 ('мочь', 441),
 ('который', 430),
 ('сам', 430),
 ('такой', 395),
 ('очень', 387),
 ('какой', 379),
 ('соня', 379),
 ('рука', 369),
 ('петрович', 369)]

In [22]:
words = word_dict.most_common()
len([item for item in words if item[1] >= 3])  # совсем мало :) 

4768

## 2. Моделирование

__Основные параметры:__

* данные должны быть итерируемым объектом 
* `size` — размер вектора, 
* `window` — размер окна наблюдения,
* `min_count` — мин. частотность слова в корпусе,
* `sg` — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* `sample` — порог для downsampling'a высокочастотных слов,
* `workers` — количество потоков,
* `alpha` — learning rate,
* `iter` — количество итераций,
* `max_vocab_size` — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

In [23]:
%%time 
from gensim.models.word2vec import Word2Vec

# size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(size=100, window=2, min_count=3, workers=4, )

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(sents_tokenize)

# обучение модели 
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке 
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов
model.train(sents_tokenize, total_examples=model.corpus_count, epochs=100)

# !NB в ситуации, когда у нас огромный корпус, 100 эпох это слишком много! 

CPU times: user 23.6 s, sys: 305 ms, total: 24 s
Wall time: 14.5 s


In [24]:
model.corpus_count # число примеров в обучающей выборке

13702

Смотрим, сколько в модели слов.

In [25]:
len(model.wv.vocab)

4768

In [26]:
'старуха' in model.wv.vocab

True

## 3. Свойства модели

In [27]:
# вектор слова
model.wv['старуха'][:10]

array([ 0.02277945,  0.59645087, -0.97010195, -0.5576507 , -0.58633286,
        0.01415113,  0.2324946 , -0.29876757,  0.8240862 ,  0.41107488],
      dtype=float32)

In [28]:
# размерность вектора
model.wv['старуха'].shape

(100,)

In [29]:
# похожести слов 
model.wv.similarity('тварь', 'право')

-0.014740856

In [30]:
model.wv.similarity('старуха', 'топор')

0.31544155

In [31]:
model.wv.similarity('тварь', 'тварь')

0.99999994

In [32]:
# самые похожие
model.wv.most_similar('старуха')

[('лизавета', 0.4834555387496948),
 ('близко', 0.4589611291885376),
 ('чиновница', 0.4112577438354492),
 ('наклепать', 0.3871508836746216),
 ('здравствуйте', 0.3868829011917114),
 ('одуматься', 0.3765152096748352),
 ('ограбить', 0.36270660161972046),
 ('снять', 0.3618197739124298),
 ('запор', 0.3613971769809723),
 ('домой', 0.3600963354110718)]

In [33]:
model.wv.most_similar('лестница')

[('прихожая', 0.5206669569015503),
 ('ступенька', 0.5133342742919922),
 ('узенький', 0.4848982095718384),
 ('всходить', 0.47868430614471436),
 ('передний', 0.47048020362854004),
 ('смутно', 0.4595457911491394),
 ('спускаться', 0.4505569338798523),
 ('ступень', 0.44437867403030396),
 ('цыпочки', 0.4261908531188965),
 ('вниз', 0.4184418320655823)]

In [34]:
# арифметика
model.most_similar(positive=['раскольников'], negative=['тварь'])[:10]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('остановиться', 0.31796663999557495),
 ('несколько', 0.29886406660079956),
 ('показаться', 0.281860888004303),
 ('уставиться', 0.2810265123844147),
 ('приятно', 0.2789955735206604),
 ('скрываться', 0.2723502516746521),
 ('провожать', 0.27022111415863037),
 ('задушить', 0.26799285411834717),
 ('испугаться', 0.25882771611213684),
 ('верно', 0.2564633786678314)]

## 4. Как дообучить модель? 

Ради чистоты эксперимента сохраним текущую модель и заново подгрузим её. 

In [35]:
model_path = "./our_w2v.model"
model.save(model_path)

In [36]:
our_model = Word2Vec.load(model_path)

Подгрузим другое произведение и сделаем для него предобработку. 

In [37]:
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/vector-school/SkazkaOCareSaltane.txt'

resp = requests.get(url)
text2 = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text2[-500:])

 узнает...
В нем взыграло ретивое!
"Что я вижу? что такое?
Как!" - и дух в нем занялся...
Царь слезами залился,
Обнимает он царицу,
И сынка, и молодицу,



И садятся все за стол;
И веселый пир пошел.
А ткачиха с поварихой,
С сватьей бабой Бабарихой
Разбежались по углам;
Их нашли насилу там.
Тут во всем они признались,
Повинились, разрыдались;
Царь для радости такой
Отпустил всех трех домой.
День прошел - царя Салтана
Уложили спать вполпьяна.
Я там был; мед, пиво пил -
И усы лишь обмочил.

1831




Предобработка.

In [38]:
text2 = text2.lower()
sents2 = sent_tokenize(text2)

sents_tokenize2 = [tokenizer.tokenize(sent) for sent in sents2]
sents_tokenize2 = [[morph.normal_forms(word)[0] for word in text_cur if word not in stopwords_ru]
                      for text_cur in sents_tokenize2]


In [39]:
sents_tokenize2[10]

['сени', 'выйти', 'царь', 'отец']

In [40]:
len(sents_tokenize2)

254

Дополняем модель.

In [41]:
# обновили словарь
our_model.build_vocab(sents_tokenize2, update=True)

# дообучили
our_model.train(sents_tokenize2, total_examples=our_model.corpus_count, epochs=100)

(177400, 269000)

In [42]:
'ядро' in model.wv.vocab

False

In [43]:
'ядро' in our_model.wv.vocab

True

In [44]:
our_model.wv.most_similar('ядро')

[('изумруд', 0.6895874738693237),
 ('грызть', 0.6602988243103027),
 ('скатерть', 0.6405527591705322),
 ('орешек', 0.6062536239624023),
 ('чистый', 0.6033691167831421),
 ('серебро', 0.5802929401397705),
 ('складка', 0.5550426244735718),
 ('научить', 0.5542415976524353),
 ('золотой', 0.5489482283592224),
 ('скорлупка', 0.5316975712776184)]

Пример со старым словом.

In [45]:
our_model.wv.most_similar('сын')

[('губерния', 0.5313253402709961),
 ('обеспечить', 0.5155653953552246),
 ('благословить', 0.5150047540664673),
 ('враньё', 0.4995172321796417),
 ('уезд', 0.4862736761569977),
 ('адвокатский', 0.4796649217605591),
 ('наущение', 0.4727262854576111),
 ('совет', 0.4673498868942261),
 ('сумление', 0.46556758880615234),
 ('раздражать', 0.46401143074035645)]

In [46]:
model.wv.most_similar('сын')

[('наущение', 0.4834819734096527),
 ('раздражать', 0.4694327116012573),
 ('совет', 0.46878790855407715),
 ('поступок', 0.4687827229499817),
 ('положение', 0.46577367186546326),
 ('интерес', 0.46447113156318665),
 ('адвокатский', 0.46211057901382446),
 ('обеспечить', 0.46016496419906616),
 ('секрет', 0.4543688893318176),
 ('сестрица', 0.45223936438560486)]

Как мы и говорили, есть еще и модель Fasttext. Есть как самостоятельная реализация в отдельной бибилиотеке (из командной строки быстрее будет учиться), так и реализация Gensim

In [47]:
from gensim.models import FastText

In [48]:
model_ft = FastText(size=100, window=3, min_count=3, iter=10)  # instantiate
model_ft.build_vocab(sentences=sents_tokenize)
model_ft.train(sentences=sents_tokenize, total_examples=len(sents_tokenize), epochs=20)

In [49]:
model_ft.wv.most_similar('старуха')

[('старухин', 0.9733899831771851),
 ('старушонка', 0.9703361988067627),
 ('старец', 0.9643481969833374),
 ('кожа', 0.9576848745346069),
 ('колея', 0.9558264017105103),
 ('старик', 0.9542369246482849),
 ('картинка', 0.9538529515266418),
 ('старичок', 0.9527403116226196),
 ('кухарка', 0.9524151086807251),
 ('старое', 0.9501200318336487)]

In [50]:
'старухин' in model_ft.wv.vocab

True

Немного развлечений

In [51]:
'старухака' in model_ft.wv.vocab

False

In [52]:
model_ft.wv.most_similar('старухака')

[('старуха', 0.9737210273742676),
 ('полечка', 0.9474987983703613),
 ('крест', 0.9360047578811646),
 ('кожа', 0.9303662776947021),
 ('старухин', 0.9291467666625977),
 ('карета', 0.9266821146011353),
 ('карта', 0.9261944890022278),
 ('полячка', 0.9254623651504517),
 ('старушонка', 0.9221905469894409),
 ('кох', 0.9221305847167969)]

In [53]:
model_ft.wv.most_similar('меч')

[('мечта', 0.9542185068130493),
 ('восьмой', 0.9344176054000854),
 ('че', 0.9257203936576843),
 ('враг', 0.9217530488967896),
 ('гораздо', 0.9176746010780334),
 ('часто', 0.917295515537262),
 ('мир', 0.9146737456321716),
 ('венец', 0.910340428352356),
 ('жестяной', 0.9101762175559998),
 ('везде', 0.9088507890701294)]

In [54]:
    model_ft.wv.similarity('старуха', 'топор')

0.8443027

А теперь возьмем что-то предобученное

In [55]:
import urllib
import gensim

In [56]:
urllib.request.urlretrieve("https://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x7f2c96b63210>)

In [57]:
model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [58]:
len(model_ru.wv.vocab)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


281776

In [59]:
model_ru.wv.most_similar('мир_S')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('вселенная_S', 0.5629245638847351),
 ('человечество_S', 0.5042722225189209),
 ('мироздание_S', 0.4957387447357178),
 ('реальность_S', 0.48456305265426636),
 ('природа_S', 0.47663506865501404),
 ('страна_S', 0.46604788303375244),
 ('действительность_S', 0.45931386947631836),
 ('царство_S', 0.45035111904144287),
 ('жизнь_S', 0.4466155171394348),
 ('бытие_S', 0.4450710713863373)]

In [60]:
model_ru.wv.most_similar('москва_S')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('петербург_S', 0.8158844709396362),
 ('ленинград_S', 0.7828081846237183),
 ('киев_S', 0.7416712641716003),
 ('ташкент_S', 0.7391982078552246),
 ('париж_S', 0.6919562816619873),
 ('петроград_S', 0.68897944688797),
 ('питер_S', 0.6885060667991638),
 ('рига_S', 0.6870608329772949),
 ('казань_S', 0.6827309131622314),
 ('одесса_S', 0.6681023836135864)]

In [61]:
model_ru.wv.most_similar('старуха_S')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('старик_S', 0.7658199071884155),
 ('старушка_S', 0.7462228536605835),
 ('баба_S', 0.5645360350608826),
 ('старушонка_S', 0.5538790225982666),
 ('бабка_S', 0.5478786826133728),
 ('тетка_S', 0.5364400148391724),
 ('нянька_S', 0.5299326777458191),
 ('мать_S', 0.5191550850868225),
 ('старичок_S', 0.5074124336242676),
 ('монашек_S', 0.49750322103500366)]

In [62]:
model_ru.wv.most_similar(positive=['старуха_S'], negative=['старик_S'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('иванна_S', 0.3401370942592621),
 ('разводиться_V', 0.3309248387813568),
 ('тюрингенский_A', 0.32313793897628784),
 ('покойница_S', 0.3211488127708435),
 ('ебеный_A', 0.31691834330558777),
 ('родить_V', 0.29508769512176514),
 ('развод_S', 0.28807705640792847),
 ('фуран_S', 0.28661054372787476),
 ('чертовый_A', 0.28630179166793823),
 ('берестовой_S', 0.28268522024154663)]

In [63]:
model_ru.wv.most_similar(positive=['королева_S'], negative=['женщина_S'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('король_S', 0.37886035442352295),
 ('франкский_A', 0.31811249256134033),
 ('аглицкий_A', 0.3177199959754944),
 ('балтасар_S', 0.3167581558227539),
 ('арканарский_A', 0.3164834976196289),
 ('пэр_S', 0.3119353652000427),
 ('арагонский_A', 0.30933094024658203),
 ('почивальня_S', 0.3064378798007965),
 ('прешпургский_A', 0.3057243227958679),
 ('фахд_S', 0.3056405782699585)]

In [64]:
model_ru.wv.most_similar(positive=['девочка_S'], negative=['маленький_S'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('девушка_S', 0.5032362937927246),
 ('мальчик_S', 0.47413814067840576),
 ('девица_S', 0.43953511118888855),
 ('девчонка_S', 0.4362611770629883),
 ('малыш_S', 0.43147242069244385),
 ('ребенок_S', 0.4179821312427521),
 ('женщина_S', 0.41777801513671875),
 ('японка_S', 0.41478800773620605),
 ('девчушка_S', 0.3969481587409973),
 ('барышня_S', 0.39466592669487)]

In [65]:
model_ru.wv.most_similar(positive=['москва_S', 'франция_S'], negative=['россия_S'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('париж_S', 0.6707262992858887),
 ('лондон_S', 0.6117687225341797),
 ('петербург_S', 0.5936024188995361),
 ('берлин_S', 0.582137405872345),
 ('ташкент_S', 0.5710321068763733),
 ('ленинград_S', 0.5650396347045898),
 ('швейцария_S', 0.564936637878418),
 ('копенгаген_S', 0.5555600523948669),
 ('мюнхен_S', 0.5477045774459839),
 ('питер_S', 0.5471242666244507)]